<p style="font-size:2em;padding-bottom: 0.5em; font-weight: bold;">
E-learning course on <br / >
Advanced Neutron Imaging
</p> 

<p style="font-size:1.25em;padding-bottom: 0.25em;">
    
# Module Number/Introduction to images

</p>

<p style="font-size:1em;">February 25, 2021</p>
<br /><br />
<p style="font-size:1.25em;padding-bottom: 0.25em; text-align: center;">Anders Kaestner</p>


<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Module-Number/Introduction-to-images" data-toc-modified-id="Module-Number/Introduction-to-images-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Module Number/Introduction to images</a></span><ul class="toc-item"><li><span><a href="#Training-objectives" data-toc-modified-id="Training-objectives-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Training objectives</a></span></li></ul></li><li><span><a href="#What-shall-we-do-with-our-images?" data-toc-modified-id="What-shall-we-do-with-our-images?-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>What shall we do with our images?</a></span><ul class="toc-item"><li><span><a href="#You-have-data..." data-toc-modified-id="You-have-data...-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>You have data...</a></span><ul class="toc-item"><li><span><a href="#Important-questions" data-toc-modified-id="Important-questions-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Important questions</a></span></li><li><span><a href="#Which-information-do-you-expect-from-the-data" data-toc-modified-id="Which-information-do-you-expect-from-the-data-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Which information do you expect from the data</a></span></li></ul></li><li><span><a href="#Measurements-are-rarely-perfect" data-toc-modified-id="Measurements-are-rarely-perfect-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Measurements are rarely perfect</a></span></li><li><span><a href="#Questions" data-toc-modified-id="Questions-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Questions</a></span></li></ul></li><li><span><a href="#Images" data-toc-modified-id="Images-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Images</a></span><ul class="toc-item"><li><span><a href="#Define-an-image" data-toc-modified-id="Define-an-image-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Define an image</a></span><ul class="toc-item"><li><span><a href="#Different-types-of-images" data-toc-modified-id="Different-types-of-images-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Different types of images</a></span></li></ul></li><li><span><a href="#Pixel-size-and-resolution" data-toc-modified-id="Pixel-size-and-resolution-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Pixel size and resolution</a></span><ul class="toc-item"><li><span><a href="#Edges-at-different-resolutions" data-toc-modified-id="Edges-at-different-resolutions-3.2.1"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span>Edges at different resolutions</a></span></li></ul></li><li><span><a href="#Image-intensity" data-toc-modified-id="Image-intensity-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Image intensity</a></span><ul class="toc-item"><li><span><a href="#The-histogram" data-toc-modified-id="The-histogram-3.3.1"><span class="toc-item-num">3.3.1&nbsp;&nbsp;</span>The histogram</a></span></li><li><span><a href="#Visualizing-gray-levels" data-toc-modified-id="Visualizing-gray-levels-3.3.2"><span class="toc-item-num">3.3.2&nbsp;&nbsp;</span>Visualizing gray levels</a></span></li><li><span><a href="#Brightness-and-contrast" data-toc-modified-id="Brightness-and-contrast-3.3.3"><span class="toc-item-num">3.3.3&nbsp;&nbsp;</span>Brightness and contrast</a></span></li></ul></li><li><span><a href="#Pixelwise-operations" data-toc-modified-id="Pixelwise-operations-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Pixelwise operations</a></span></li><li><span><a href="#Questions" data-toc-modified-id="Questions-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Questions</a></span></li></ul></li><li><span><a href="#Noise-in-images" data-toc-modified-id="Noise-in-images-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Noise in images</a></span><ul class="toc-item"><li><span><a href="#Questions" data-toc-modified-id="Questions-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Questions</a></span></li></ul></li><li><span><a href="#Summary" data-toc-modified-id="Summary-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Summary</a></span></li></ul></div>

## Training objectives

- Terminal Training Objectives:
    - Obtain an awareness that numerical methods are needed for the quantitative interpretation of images.
    - Understand the concept of an image processing workflow.

- Enabling training objectives:
    - Understanding that the analysis objectives decide the choice of strategy and tools.
    - Basic definitions of 
        - images
        - gray levels
        - visibility
        - noise


# What shall we do with our images?

## You have data...

Imaging experiments produce large amounts of data

![Experiment data](figures/experimentdata.svg)

### Important questions

What is the purpose of the experiment?

- Qualitative visual analysis using 3D visualization
- Sample characterization
- Determine process parameters
- Count and measure size of features
- etc.



### Which information do you expect from the data


| Intensity  | Geometry |
|:----------|:----------|
| Material composition | Identify features |
| Material transport   | Volume |
| Physical quantities | Shape |

The information you want to quantify affects:
- The choice of processing method
- The experiment strategy 
- The choice of analysis tools


## Measurements are rarely perfect

## Questions

# Images

## Define an image

### Different types of images

## Pixel size and resolution 

### Edges at different resolutions


## Image intensity

### The histogram

### Visualizing gray levels

### Brightness and contrast

## Pixelwise operations

## Questions

# Noise in images

## Questions

# Summary 
